In [1]:
import pandas as pd
import numpy
import requests
import acquire
import prepare
import os
from sklearn.model_selection import train_test_split

In [2]:
df = acquire.get_competition_data()

In [ ]:
df = prepare.prepare_competition_data(df)

In [ ]:
df.info()

In [ ]:
#steps for prepare

In [4]:
df = df.drop(columns=['QA', 'QA.1', 'QB', 'QB.1', 'PR', 'PR.1', 'Nat_x', 'Nat_y', 'season_x', 'season_y', '#_x', '#_y'])
df['SP'] = df.SP.fillna('no_best')
df['FS'] = df.FS.fillna('no_best')

In [ ]:
#df[['skater_name','event','Unnamed: 0']]

In [5]:
df['Solo Jump'] = df['Solo Jump'].fillna('3Lz')

In [6]:
df['Total'] = df['Total'].fillna('no_best')

In [7]:
df = df.dropna()

In [ ]:
#df = df.drop(columns=['Nat_x', 'Nat_y', 'season_x', '#_x', '#_y', 'TSS_x', 'TSS_y'])

In [8]:
df.rename(columns={'Skater':'last_name', 'SP': 'short_best', 'SP.1': 'short_score', 'SP.2': 'short_place', 'Total': 'event_best', 'Total.1': 'event_score', 'Unnamed: 0': 'event_place',  'Combo Jump': 'short_combo_jump', 'Solo Jump': 'short_solo_jump', 'Axel': 'short_axel_jump', 'TES_x': 'short_elements_score', 'TES.1_x': 'short_elements_rank', 'PCS_x': 'short_components_score', 'PCS.1_x': 'short_components_rank', 'TES_y': 'free_elements_score', 'TES.1_y': 'free_elements_rank', 'PCS_y': 'free_components_score', 'PCS.1_y': 'free_components_rank', 'FS': 'free_best', 'FS.1': 'free_score', 'FS.2': 'free_place', 'Nat': 'country', 'Elements': 'free_elements', 'TSS_x': 'short_overall', 'TSS_y': 'free_overall'}, inplace=True)

In [ ]:
#'FS': 'free_best', 'FS.1': 'free_score', 'FS.2': 'free_place', 'Nat': 'country', 'Elements': 'free_elements'

In [9]:
df['short_best'] = df['short_best'].replace({'PB': 'skater_best', 'SB': 'skater_season_best'})
df['free_best'] = df['free_best'].replace({'PB': 'skater_best', 'SB': 'skater_season_best'})

In [10]:
olympic_skaters = df.skater_name[df.event == 'olympics'].unique()
df['olympian'] = df.apply(lambda x: True if x.skater_name in olympic_skaters else False, axis=1)

In [21]:
df_oly = df[df['olympian'] == True]

In [ ]:
df_oly

In [12]:
df_oly = df_oly.replace({'<<':'@'}, regex=True)

In [24]:
#df_oly['free_elements'] = df_oly.loc[:, ('free_elements')].replace({'x':'#'})

/var/folders/s4/hmz6ljm533vgpm_bhv59yw0m0000gn/T/ipykernel_75747/434371037.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oly['free_elements'] = df_oly.loc[:, ('free_elements')].replace({'x':'#'})


In [14]:
df_oly['free_deductions'] = (df_oly['free_overall'].str[1:2]).astype(int)
df_oly['short_deductions'] = (df_oly['short_overall'].str[1:2]).astype(int)
df_oly['event_deductions'] = df_oly['short_deductions'] + df_oly['free_deductions']

In [15]:
df_oly['free_elements'] = df_oly['free_elements'].replace({'q':'~'}, regex=True)

In [ ]:
df_oly.short_combo_jump.tail(20)

In [16]:
df_oly['free_elements'] = df_oly['free_elements'].replace({'e':'$'}, regex=True)
df_oly['short_combo_jump'] = df_oly['short_combo_jump'].replace({'e':'$'}, regex=True)
df_oly['short_axel_jump'] = df_oly['short_axel_jump'].replace({'e':'$'}, regex=True)
df_oly['short_solo_jump'] = df_oly['short_solo_jump'].replace({'e':'$'}, regex=True)

In [ ]:
df_oly.free_elements

In [17]:
df_oly['under_rotations'] = df_oly.apply(lambda x: x.str.contains("\<"), axis=1).sum(axis=1)
df_oly['downgrades'] = df_oly.apply(lambda x: x.str.contains("\@"), axis=1).sum(axis=1)
df_oly['time_bonus_elements'] = df_oly.apply(lambda x: x.str.contains("#"), axis=1).sum(axis=1)
df_oly['edge_error'] = df_oly.apply(lambda x: x.str.contains("\$"), axis=1).sum(axis=1)
df_oly['suspected_edge_error'] =  df_oly.apply(lambda x: x.str.contains("\!"), axis=1).sum(axis=1)
df_oly['suspected_rotation_error'] = df_oly.apply(lambda x: x.str.contains('\~'), axis=1).sum(axis=1)
df_oly['combo_no_credit'] =  df_oly.apply(lambda x: x.str.contains("\+COMBO"), axis=1).sum(axis=1)
df_oly['illegal_element'] =  df_oly.apply(lambda x: x.str.contains("\*"), axis=1).sum(axis=1)


In [19]:
df_oly['under_rotations'].value_counts()

0.0    542
1.0    380
2.0    174
3.0     30
4.0      1
Name: under_rotations, dtype: int64

In [ ]:
df_oly['time_bonus_elements'] = df_oly.apply(lambda x: x.str.contains("##"), axis=1).sum(axis=1)

In [ ]:
df_oly['under_rotations'] = df_oly['under_rotations'].astype(int)
df_oly['downgrades'] = df_oly['downgrades'].astype(int)
df_oly['time_bonus_elements'] = df_oly['time_bonus_elements'].astype(int)
df_oly['edge_error'] = df_oly['edge_error'].astype(int)
df_oly['suspected_edge_error'] =  df_oly['suspected_edge_error'].astype(int)
df_oly['suspected_rotation_error'] = df_oly['suspected_rotation_error'].astype(int)
df_oly['combo_no_credit'] =  df_oly['combo_no_credit'].astype(int)
df_oly['illegal_element'] =  df_oly['illegal_element'].astype(int)
df_oly['event_deductions'] = df_oly['event_deductions'].astype(int)

In [ ]:
df_oly['jump_errors_all'] = df_oly['under_rotations'] + df_oly['downgrades'] + df_oly['edge_error'] + df_oly['suspected_edge_error'] + df_oly['suspected_rotation_error'] + df_oly['combo_no_credit'] + df_oly['illegal_element'] + df_oly['event_deductions']

In [ ]:
df_oly['jump_errors_costly'] = df_oly['downgrades'] + df_oly['combo_no_credit'] + df_oly['event_deductions']

In [ ]:
df_oly.country.dtype

In [ ]:
df_oly['country_flag'] = df_oly['country']

In [ ]:
df_oly['country'] = df_oly.country_flag.replace({'🇯🇵': 'japan', '🇺🇸': 'usa', '🇷🇺': 'russia', '🇨🇦': 'canada',
                                           '🇰🇷': 'south_korea', '🇮🇹': 'italy', '🇨🇳': 'china',
                                            '🇫🇮': 'finland', '🇭🇺': 'hungary', '🇬🇪': 'georgia',
                                            '🇩🇪': 'germany', '🇨🇭':'switzerland', '🇫🇷':'france',
                                            '🇦🇺': 'australia', '🇪🇪': 'estonia', '🇰🇿':'kazakhstan',
                                            '🇺🇦': 'ukraine', '🇺🇿': 'uzbekistan', '🇸🇰': 'slovakia',
                                            '🇭🇷': 'croatia', '🇹🇷': 'turkey', '🇦🇿': 'azerbaijan',
                                            '🇧🇪': 'belgium', '🇨🇿':'czech_republic', '🇪🇸': 'spain',
                                            '🇵🇱': 'poland', '🇦🇹': 'austria', '🇧🇾': 'belarus',
                                            '🇧🇬': 'bulgaria', '🇧🇷': 'brazil', '🇱🇺': 'luxembourg'})

In [ ]:
df_oly['country'].value_counts()

In [ ]:
df_oly.info()

In [ ]:
train, test = train_test_split(df_oly, test_size=.2, random_state=123)

In [ ]:
train.event.value_counts()

In [ ]:
train.free_elements.value_counts()

In [ ]:
train.dtypes